# CS341 Project 4: Genetic Algorithms

Your first goal is to analyze the performance of several genetic algorithms in the context of parameter-estimation. Your second goal is to analyze the results, i.e. what do we learn about the model? In this project, we are revisiting Goldbeter's 5-state fly clock model, whose parameters were originally chosen "by hand." You will use a cost function which ensures the oscillations in constant darkness have a period of 23.6 hours.

There are many, many variants of genetic algorithms. We will be using the following algorithm.

<ul>
    <li>Create an initial population of $\lambda$ individuals $G^0$ and generate their costs.
    <li>Sort the individuals in $G^0$ by cost (in preparation for selection)
    <li>For each generation $g$
        <ul>
        <li>Take the $eliteCount$ best children from the previous generation $G^{g-1}$ and put them into this generation $G^g$
            <li><b>Select</b> the breeding pool $P$ of $\mu$ individuals from the previous generation.
        <li>For i in range($eliteCount$,$\lambda$)
            <ul>
            <li>Randomly choose two parents from $P$.
            <li>Use <b>cross-over</b> to generate a child $G^g_i$.
            <li><b>Mutate</b> the values in $G^g_i$.
            <li>Compute the child's cost.
            </ul>
        <li>Sort the individuals in $G^g$ by cost (in preparation for selection)
        </ul>
</ul>   

The key operators and parameters can be varied:
<ul>
    <li>Selection can favor fit (low-cost) parents more or less, and always has an aspect of randomness. If we favor low-cost parents more, then the algorithm converges quickly, but it doesn't explore parameter space very well, and could miss a more fit individual. If we favor them less, the algorithm might not find children of increasingly good fitness. Explanations of several selection operators may be found in: Blickle, T. and Thiele, L. 1995. A comparison of selection schemes used in genetic algorithms. Tech. Rep. TIK-Report 11, Swiss Federal Institute of Technology (ETH), Zurich, Switzerland, May.
    <li>Crossover is typically "uniform" (each element of the child is randomly taken from one of its two parents) or "single" (a cross-over point is selected and all elements up to the cross-over point are taken from one parent and the remaining elements are taken from the second parent).
    <li>Mutation is typically implemented by choosing a value from a Gaussian distribution centered on each element. One straightforward way to vary the mutation is to vary the width of the distribution. We control the size of the mutation by scaling the width of the Gaussian distribution (it should be $mutationScale\cdot parameterValue$).
    <li>The number of parents $\mu$ and children $\lambda$ affects how widely we can sample space. Large numbers of children allow for a broader sampling. The number of parents (the size of the breeding pool) should be a fraction of the number of children (typically, it is a number like 1/5). It works with the selection operator to control how broadly the space is searched and how quickly the algorithm progresses towards a solution. The number of children that you need to use depends upon the problem. It can range from in the tens (e.g. our simple island cost function from class) to thousands.
    <li>The number of elite individuals should be small (e.g. 1 to 3). Including elites guarantees that the best-fit individual in generation $g$ will be at least as fit as the best-fit individual in generation $g-1$.
    <li>The number of generations should be as big as it needs to be for the best (or average) cost to stop improving. For some problems, thousands of generations are used. Here, we are able to use many fewer (e.g. I used 5 for the simple island cost function in class)
</ul>  

## Algorithm Performance

<ol>
<li> Run each algorithm on the fly model at least 5 times for each ``flavor'' for at least 5 generations,  i.e. analyze performance for 
	<ol>
    <li> a GA with truncation selection, an elite count of 1, 10 parents, 50 children, and a mutation scale of 0.05
	<li> a GA with tournament selection (tournament size = 2), an elite count of 1, 10 parents, 50 children, and a mutation scale of 0.05
	<li> a GA with linear ranking selection, an elite count of 1, 10 parents, 50 children, and a mutation fraction of 0.05
    </ol>
<li> Report your results in a concise, but informative manner. You will want to identify trends. To do this, quantify the performance of the algorithm. For example, determine the mean population cost for each generation and plot how that changes across generations.
</ol>

## Implication for Model
Using some of your output from above, analyze the effects of different parameters on the model's performance. Below are suggestions to guide your analysis. Follow at least one of them.

<ol>
<li> How different are the optimization results? i.e. how different are the sets of model parameters (e.g. rate constants and activation thresholds) that are found by different runs of the GA?  
<li> You may see different relationships between amplitudes of clock components. Do you see any patterns in this behavior? (e.g. is the peak mRNA concentration always bigger (or always smaller) than the peak P1 concentration?) If so, maybe you can conclude that the relative amplitude is not a property of the model's parameters, but that it is a property of its structure. Make a detailed case for your conclusion.
<li> Choose several optimization results and determine whether or not the non-intuitive behavior mentioned in Goldbeter's paper occurs. In his paper, he showed that increasing the rate of Per protein degradation ($v_d$) increases the total PER in the system. Is this true for multiple optimization results (i.e. model parameter sets)?
</ol>

## Extensions

<ul>
<li> Implement proportional selection and include analysis of the GA with proportional selection it in your write-up.
<li> Include additional analysis with different values of the algorithm's parameters. What happens if the number of children per generation is different? What about increasing or decreasing the mutation fraction?
<li> Include simulated annealing in your set of optimization algorithms.
<li> Implement a simulated annealing optimization algorithm.
<li> Make a hybrid algorithm that uses both a GA (or simulated annealing) and a deterministic method (e.g. a hill-descending method). The idea is that the stochastic algorithms helps you find the right region of parameter space, and that in that region, the cost function will be smooth (and maybe even monotonic). Once we are in a region that is smooth and monotonic, we can use a deterministic optimization method to refine our parameter set to find the local optimum.
<li> Include amplitude in the cost function for Goldbeter's model: add a penalty for each state variable whose peak-to-trough amplitude is less than 0.1.
<li> Also find parameters for the Gonze/Goodwin oscillator.
</ul>


## Steps to get you started
### 1. Cost function for fly clock model

To write the cost function for the fly clock model, you will need gol95_model and get_period from project 2. 

Your cost function should have the signature:
<code>
def gol95_cost( params ):    
</code>

where params is the ndarray of parameters used to simulate the model.

It should
<ul>
    <li>Run the simulation with params as the parameters for at least 10 days, so that it is likely to have reached the limit cycle.
    <li>Re-run the simulation, beginning with the values from the final time step of the previous simulation.
    <li>Compute the period $per$ and the cycle-to-cycle standard deviation of the period $sdper$ by calling get_period.
    <li>Compute the cost according to
        $\sqrt{ \left(\frac{per-23.6}{23.6}\right)^2 + \frac{sdper}{23.6}}$
</ul>


In [5]:
# Write your code here (copy-paste the model and get_period, write gol95_cost)
import scipy
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# model
def gol_95(t,y,params):
    '''

    '''

    # Unpacking Params(18)
    v_s = params[0]
    v_m = params[1]
    K_m = params[2]
    k_s = params[3]
    v_d = params[4]
    k_1 = params[5]
    k_2 = params[6]
    K_I = params[7]
    K_d = params[8]
    n = params[9]
    K_1 = params[10]
    K_2 = params[11]
    K_3 = params[12]
    K_4 = params[13]
    V_1 = params[14]
    V_2 = params[15]
    V_3 = params[16]
    V_4 = params[17]

    #Unpacking Y
    M = y[0]
    P_0 = y[1]
    P_1 = y[2]
    P_2 = y[3]
    P_N = y[4]

    #Calculating the 5 differential equations
    dM_dt = v_s * K_I^n/(K_I^n + P_N^n)  -  v_m * M / (K_m + M)
    dP_0_dt = k_s * M  -  V_1*P_0/(K_1+P_0)  +  V_2 * P_1/(K_2+P_1)
    dP_1_dt =  V_1*P_0/(K_1+P_0) - V_2 * P_1/(K_2+P_1) - V_3 * P_1/(K_3+P_1) + V_4*P_2/(K_4+P_2)
    dP_2_dt = V_3 * P_1/(K_3+P_1) - V_4*P_2/(K_4+P_2) - k_1*P_2 + k_2*P_N - v_d * P_2/(K_d+P_2)
    dP_N_dt = k_1*P_2 - k_2*P_N

    return (dM_dt,dP_0_dt,dP_1_dt,dP_2_dt,dP_N_dt)

In [3]:
def get_period(t,x):
     """ Approximate the period of a 1-D x, given the time-steps t.
         Returns a tuple with the period and the standard deviation of
the period over time.
         if the value of the standard deviation is not smaller than 0.1,
then
         it means the period estimate is dodgy and you shouldn't use it.
Instead,
         plot your simulation and figure out why it isn't periodic -
maybe it just
         hasn't reached the limit cycle yet."""
     idxs = scipy.signal.find_peaks(x)
     idxs = idxs[0]
     times = t[idxs]
     periods = np.diff(times)
     period = periods.mean()
     sdperiod = periods.std()

     return (period,sdperiod)

In [4]:
# Cost function
def gol95_cost( params ):
    '''

    :param params: the parameters for the gol_95 model
    :return: cost: a float that represents how off the gol95 simulation is off from the expected results
    '''




    period = 1
    sdperiod = 1

    cost = (((period - 23.6)/23.6)^2 + sdperiod/23.26)^0.5

    return cost

In [7]:
# Test your cost function here.
# When Stephanie tests hers with the parameters from Project 2, her cost is < 0.05

# Parmaters for the model
# Setting up the gol_95 parameters
v_s = 0.76
v_m = 0.65
K_m = 0.5
k_s = 0.38
v_d = 0.95
k_1 = 1.9
k_2 = 1.3
K_I = 1
K_d = 0.2
n =   4
K_1 = 2
K_2 = 2
K_3 = 2
K_4 = 2
V_1 = 3.2
V_2 = 1.58
V_3 = 5
V_4 = 2.5
gol_95_params = (v_s,v_m,K_m,k_s,v_d,k_1,k_2,K_I,K_d,n,
                 K_1,K_2,K_3,K_4,V_1,V_2,V_3,V_4)


# Setting up the inputs
M = 1
P_0 = 1
P_1 = 1
P_2 = 1
P_N = 1

gol_95_inputs = (M, P_0, P_1,P_2, P_N)

### 2. Selection operator

Write your selection operators in this cell, so that you can test them with artificial data. I have included a barebones operator along with test code below. The uniform operator is a useless one, because it applies no selection pressure. The test code has costs ranging from 0.1 to 500 and the uniform operator samples from them all, so the histogram of costs from the breeding pool should indicate a uniform distribution from 0.1 to 500. 

In [2]:
import matplotlib.pyplot as plt
import numpy as np

class GASettings:
    def __init__( self, numParents = 10, numChildren = 50 ):
        self.numParents = numParents;
        self.numChildren = numChildren;
        self.numGenerations = 5;
        self.selection = 'truncation'; 
        self.slope = 15; # for linear ranking selection
        self.tournamentSize = 2;
        self.crossover = 'uniform';
        self.mutation = 0.05; # std of normal distribution computed as mutation*value
        self.eliteCount = 1;

def select_pool( G, Gcost, settings ):
    """ Select a breding pool from the previous generation.
    G is numChildren x numParameters
    Gcost is numChildren long and is sorted in ascending order.
    Returns (P,Pcost)
    where 
    P is numParents x numParameters
    Pcost is numParents long.
    """
    if Gcost.size != settings.numChildren:
        print("problem")
        return
    P = np.zeros( (settings.numParents,G.shape[1]) )
    Pcost = np.zeros( (settings.numParents,) )

    if settings.selection == 'uniform':
        for i in range(settings.numParents):
            idx = np.random.choice(G.shape[0])
            P[i,:] = G[idx,:]
            Pcost[i] = Gcost[idx]
    else:
        print( "Unknown selection operator: ", settings.selection )
    return (P, Pcost)

# Test the selection operators
# Make fake parameters, using the index as the values (so we can differentiate them)
lam = 10000 # number of individuals in generation
mu = 1000 # number of individuals in breeding pool
NP = 5 # num of parameters per individual
G = np.zeros( (lam,NP) )
for i in range(lam):
    G[i,:] = i
Gcost = np.linspace(0.1,500,lam)
settings = GASettings(numParents=mu, numChildren=lam)

settings.selection = 'uniform'
(P, Pcost) = select_pool(G, Gcost, settings)
plt.figure( figsize=(8,4) )
plt.hist( Pcost );
plt.title( "Breeding Pool Chosen Uniformly")
plt.xlabel( 'Cost')
plt.ylabel('Frequency')



ModuleNotFoundError: No module named 'matplotlib'

### 3. Test cross-over and mutation

Stephanie puts cross-over and mutation in a function named generateChild. The input to generateChild is the cost function, the breeding pool, the lower bounds, the upper bounds, and the settings. You can test it by generating an artificial breednig pool, such as one in which the first individual is an array of all 1's, the second is all 2's etc. Then, if you print out the results, which two individuals were chosen, how the cross-over was done and how the mutation worked should be clear.

In [ ]:
# Write your code for testing cross-over and mutation here.

### 4. Write your Genetic Algorithm

Write it according to the algorithm above and what we have done in class.

Test it with the simple island cost function from class.

In [ ]:
# Your GA code goes here! Copy-paste the simple island cost function from the Jupyter notebook from class.


### 5. Run your Genetic Algorithm on the Fly Clock Model

Below, I supply code that generates the upper and lower bounds for the parameters. The rest is up to you.

In [ ]:
# Code with lower and upper bounds the parameters in Goldbeter's fly clock model.
# I put the published values in comments.

lb = np.zeros((18,));
ub = np.zeros((18,));
lb[0] = 0;   ub[0] = 1;# vs = 0.76;
lb[1] = 0;   ub[1] = 1;# vm = 0.65;
lb[2] = .1;  ub[2] = 1;# Km = 0.5;
lb[3] = 0;   ub[3] = 1;# ks = 0.38;
lb[4] = 0;   ub[4] = 1;# vd = 0.95;
lb[5] = 0;   ub[5] = 2;# k1 = 1.9;
lb[6] = 0;   ub[6] = 2;# k2 = 1.3;
lb[7] = 1;   ub[7] = 1;# KI = 1;
lb[8] = 0.1; ub[8] = 1;# Kd = 0.2;
lb[9] = 4;  ub[9] = 4;# n = 4;
lb[10] = 0.5;  ub[10] = 2.5;# K1 = 2;
lb[11] = 0.5;  ub[11] = 2.5;# K2 = 2;
lb[12] = 0.5;  ub[12] = 2.5;# K3 = 2;
lb[13] = 0.5;  ub[13] = 2.5;# K4 = 2;
lb[14] = 0;  ub[14] = 5;# V1 = 3.2;
lb[15] = 0;  ub[15] = 5;# V2 = 1.58;
lb[16] = 0;  ub[16] = 5;# V3 = 5;
lb[17] = 0;  ub[17] = 5;# V4 = 2.5;


   ## Begin your project here
   
  Complete the project (analyzing the algorithm's performance and the parameters output) here. You can add your new selection operators to the code in cells above. Just concentrate main code for the report here.


In [10]:
# Start code here and write-up here. :)

<hr style="border:2px solid gray"> </hr>

Acknowledgements

*Replace this text with links to external resources and thanks to people you worked with.*